# Analysis of a CFIS shear catalogue
First steps. Analysing both ShapePipe and Lensfit catalogues, for all blinds A,B and C

In [6]:
import sys
import numpy as np
from astropy.io import fits
import matplotlib.pylab as plt
import treecorr
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
# import pyccl as ccl

%matplotlib inline
plt.rcParams.update({'font.size': 20,'figure.figsize':[12,10]})
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='sans-serif')

#SPECIFY DIRECTORIES AND CATALOGUE PATHS
work_dir = '/home/mkilbing/astro/data/UNIONS/v1.x/ShapePipe'

cat_dict = {1:{'dir': work_dir + '/Lensfit/lensfit_goldshape_2022v1.fits',
                  'label': 'LF_full',
                  'e1_bias': 0,
                  'e2_bias': 0,
                  'ls': 'solid',
                  'colour': 'g'},
           2:{'dir': work_dir + '/ShapePipe/unions_shapepipe_2022_v1.0.fits',
                 'label': 'SP_full',
                 'e1_bias':0,
                 'e2_bias':0,
                  'ls': 'solid',
                  'colour': 'b'},
           3:{'dir': work_dir + '/matched_LF_SP/masked_matched_lensfit_goldshape_2022v1.fits',
                  'label': 'LF_matched_SP',
                  'e1_bias': 3.939e-4,
                  'e2_bias': 6.482e-5,
                  'ls': 'dotted',
                  'colour': 'g'},
           4:{'dir': work_dir + '/matched_LF_SP/masked_matched_unions_shapepipe_extended_2022_v1.0.fits',
                  'label': 'SP_matched_LF',
                  'e1_bias':-5.6726e-5,
                  'e2_bias':8.218e-4,
                  'ls': 'dotted',
                  'colour': 'b'},
           5:{'dir': work_dir + '/matched_LF_SP/matched_footprint_shapepipe.fits',
                  'label': 'SP Match LF Footprint',
                  'e1_bias':0,
                  'e2_bias':0,
                  'ls': 'dashed',
                  'colour': 'b'},
           6:{'dir': work_dir + '/cfis-shapepipe.parquet',
                  'label': 'SP Match MegaPipe',
                  'e1_bias':0,
                  'e2_bias':0,
                  'ls': 'dashdot',
                  'colour': 'b'},
          7:{'dir': work_dir + '/ShapePipe/shapepipe_1500_goldshape_v1.fits',
                  'label': 'SP_1500',
                  'e1_bias':7.156105098141909e-06,
                  'e2_bias':-6.00816359759969e-06,
                  'ls': 'dotted',
                  'colour': 'b'},
          8:{'dir': work_dir + '/ShapePipe/unions_shapepipe_2022_v1.0.4.fits',
                  'label':'SP_cut_Fabian',
                  'e1_bias':0.0,
                  'e2_bias':0.0,
                  'ls': 'dashdot',
                  'colour': 'pink'},
          9:{'dir': work_dir + '/ShapePipe/unions_shapepipe_psf_2022_v1.0.2.fits',
                 'label':'SP_PSF',
                  'e1_bias':0.0,
                  'e2_bias':0.0,
                  'ls': 'dashdot',
                  'colour': 'b'},
          10:{'dir': work_dir + '/unions_shapepipe_2022_v1.3.fits',
                 'label':'SP_v1.3',
                  'e1_bias':0.0,
                  'e2_bias':0.0,
                  'ls': 'dashdot',
                  'colour': 'r'},
          11:{'dir': work_dir + '/ShapePipe/unions_shapepipe_star_2022_v1.3.fits',
                 'label':'SP_v1.3',
                  'e1_bias':0.0,
                  'e2_bias':0.0,
                  'ls': 'dashdot',
                  'colour': 'b'},
          12:{'dir': work_dir + '/unions_shapepipe_2024_v1.4.1.fits',
                 'label':'SP_v1.4.1',
                  'e1_bias':0.0,
                  'e2_bias':0.0,
                  'ls': 'dashdot',
                  'colour': 'b'}}

In [ ]:
# CATALOGUE OPTIONS: 
# 1: LensFit Full
# 2: ShapePipe Full
# 3: LF Match SP
# 4: SP Match LF
# 5: SP Matched in LF footprint
# 6: SP Match MegaPipe
# 7: SP 1500deg2 (Axel's)
# 8: SP cut on large gals
# 12: SP psfex v1.4.1

cat_options=[10,12]

dfs = []

for cat_option in cat_options:
    if cat_option==6:
        df = pd.read_parquet(cat_dict[cat_option]['dir'], engine='pyarrow')
        df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    else:
        with fits.open(cat_dict[cat_option]['dir']) as data:
            df = pd.DataFrame(data[1].data)
        if cat_option==7:
            df = df.rename(columns={'g1':'e1','g2':'e2'})
        if cat_option==8 or cat_option==10:
            df = df.rename(columns={'RA':'ra','Dec':'dec'})
        if cat_option==12:
            df = df.rename(columns={'RA':'ra','Dec':'dec','e1':'e1_prev','e2':'e2_prev'})
            df = df.rename(columns={'e1_noleakage':'e1','e2_noleakage':'e2'})
    dfs.append(df)

## Catalogue Analysis

In [ ]:
for cat in cat_options:
    plt.plot(df['ra'],df['dec'], '.',label=cat_dict[cat]['label'])
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')
plt.legend(loc='upper right')
# plt.savefig('plots/3500deg^2_plot.pdf',dpi=100)
plt.show()

In [ ]:
#Ellipticity histograms
plt.rcParams.update({'font.size': 20,'figure.figsize':[22,7]})

fig, axs = plt.subplots(1, 2)
nbins = 200

for idx,cat in enumerate(cat_options):
    (n,bins,_)= axs[0].hist(dfs[idx]['e1'], bins=nbins, density=True, histtype='step',
                            weights=dfs[idx]['w'],label='e1 %s' %cat_dict[cat]['label'])
axs[0].set_xlabel(r'$e_1$')
axs[0].legend()
axs[0].set_xlim([-1.5,1.5])

# axs[0].set_ylim([0,2e4])

for idx,cat in enumerate(cat_options):
    (n,bins,_)= axs[1].hist(dfs[idx]['e2'], bins=nbins, density=True, histtype='step',
                            weights=dfs[idx]['w'],label='e2 {}'.format(cat_dict[cat]['label']))
    print('e1 sigma {}: {}'.format(cat_dict[cat]['label'],np.std(dfs[idx]['e1_noleakage'])))
    print('e2 sigma {}: {}'.format(cat_dict[cat]['label'],np.std(dfs[idx]['e2_noleakage'])))
    print('e1 bias {}: {}'.format(cat_dict[cat]['label'],np.average(np.array(dfs[idx]['e1_noleakage']), weights=np.array(dfs[idx]['w']))))
    print('e2 bias {}: {}'.format(cat_dict[cat]['label'],np.average(np.array(dfs[idx]['e2_noleakage']), weights=np.array(dfs[idx]['w']))))
axs[1].set_xlabel(r'$e_2$')
axs[1].legend()
axs[1].set_xlim([-1.5,1.5])
# axs[1].set_ylim([0,2e4])


In [ ]:
#Mag histograms

plt.rcParams.update({'font.size': 20,'figure.figsize':[15,10]})

for idx,cat in enumerate(cat_options):
    (n,bins,_)= plt.hist(dfs[idx]['mag'], bins=200, density=False, histtype='step', 
                            weights=dfs[idx]['w'],label='Mag %s' %cat_dict[cat]['label'])
    
plt.xlim([19,26])
plt.xlabel('Mag')
plt.legend(loc='upper left')


## Plot n(z)'s from file


In [ ]:
# nz_lf = fits.open(work_dir + '/nz/blind_nz_cfis_lensfit_goldshape_2022v1.fits')[1].data
nz = fits.open(work_dir + '/nz/blind_nz_cfis_shapepipe_2022v1.fits')[1].data

# nz_lf_matched = fits.open(work_dir + '/nz/nz_masked_matched_lensfit_goldshape_2022v1.fits')[1].data
# nz_sp_matched = fits.open(work_dir + '/nz/nz_masked_matched_unions_shapepipe_extended_2022_v1.0.fits')[1].data



In [ ]:
#FULL CATALOGUE NZ'S
from matplotlib.ticker import StrMethodFormatter

blinds = ['A','B','C']

# for blind in blinds:
#     z1 = nz_lf['Z_%s' %blind]

#     (n,bins,_)= plt.hist(z1, bins=200, range=(0,5.0), density=True, histtype='step', weights=None,label='LensFit Blind %s' %blind)
#     # n_lf.append(list(n))
#     # bins_lf.append(list(bins))

#     plt.xlabel('Redshifts')
#     plt.ylabel('n(z)')
#     print("zmin = ",min(z1))
#     print("zmax = ",max(z1))
#     plt.legend(fontsize=20)
#     # plt.savefig('plots/Lensfit_nz_all_blinds.pdf' )
# plt.show()
    #####################################################################################################
for blind in blinds:
    z = nz['Z_%s' %blind]
    bins=np.linspace(0,5,100)
    
    y,edges = np.histogram(z, bins,density=True,weights=nz['som_w'])
    centers = 0.5*(edges[1:]+ edges[:-1])
    plt.plot(centers,y,'-o',markersize=4,label='Blind %s' %blind,alpha=0.7)

    # (n,bins,_)= plt.hist(z2, bins=50, range=(0,5.0), density=True, histtype='step',weights=nz['som_w'],label='Blind %s' %blind,alpha=0.5)
    # n_sp.append(list(n))
    # bins_sp.append(list(bins))

    plt.xlabel(r'$z$')
    plt.ylabel(r'$n(z)$')
    plt.ylim([0,1.7])
    plt.xlim([0,5])
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.1f}'))
    # print("zmin = ",min(z))
    # print("zmax = ",max(z))
    plt.legend(fontsize=20)
    plt.savefig('../plots/unions_nz.pdf',bbox_inches='tight')
plt.show()



## Compute shear-shear correlation

In [ ]:
# Create TreeCorr catalogue
treecorr.set_omp_threads(8)

sep_units = 'arcmin'
theta_min = 1
theta_max = 200

TreeCorrConfig = {
        'ra_units': 'degrees',
        'dec_units': 'degrees',
        'max_sep': str(theta_max),
        'min_sep': str(theta_min),
        'sep_units': sep_units,
        'nbins': 20,
        'var_method':'jackknife',
    }

cat_ggs = []
for idx,cat in enumerate(cat_options):
    cat_gal = treecorr.Catalog(
        ra=dfs[idx]['ra'],
        dec=dfs[idx]['dec'],
        g1=dfs[idx]['e1']-cat_dict[cat]['e1_bias'],
        g2=dfs[idx]['e2']-cat_dict[cat]['e2_bias'],
        w=dfs[idx]['w'],
        ra_units='degrees',
        dec_units='degrees',
        npatch=50
    )
    gg = treecorr.GGCorrelation(TreeCorrConfig)
    gg.process(cat_gal)
    cat_ggs.append(gg)
    print("done for cat %s" %cat)

In [ ]:
plt.rcParams.update({'font.size': 25,'figure.figsize':[10,7]})

ax1 = plt.subplot(111)
for idx,cat in enumerate(cat_options):
    ax1.plot(cat_ggs[idx].meanr, cat_ggs[idx].npairs, 
             label=r'$n_{pairs}$ %s' %(cat_dict[cat]['label']),
             ls=cat_dict[cat]['ls'],color=cat_dict[cat]['colour'])
ax1.set_xlabel(rf'$\theta$ [{sep_units}]')
ax1.set_ylabel(r'$n_{pairs}$')
plt.show()

ax2 = plt.subplot(111)
for idx,cat in enumerate(cat_options):
    ax2.errorbar(cat_ggs[idx].meanr, cat_ggs[idx].xip, yerr=np.sqrt(cat_ggs[idx].varxip), 
                 label=r'$\xi_+$ %s' %(cat_dict[cat]['label']),
                 ls=cat_dict[cat]['ls'],color=cat_dict[cat]['colour'])
    ax2.axvspan(0,10,color='gray', alpha=0.3)
    # ax2.axvspan(100,200,color='gray', alpha=0.3)
    
ax2.text(0.85, 0.88, '1,1', transform=ax2.transAxes,
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round', pad=0.5))
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel(rf'$\theta$ [{sep_units}]')
ax2.set_xlim([0,200])
_ = ax2.set_ylabel(r'$\xi_+(\theta)$')
ax2.legend(loc='lower left')
# plt.savefig('../plots/xi_plus_%s.pdf' %cat_dict[cat]['label'],bbox_inches='tight')
plt.show()

ax3 = plt.subplot(111)
for idx,cat in enumerate(cat_options):
    ax3.errorbar(cat_ggs[idx].meanr, cat_ggs[idx].xim, yerr=np.sqrt(cat_ggs[idx].varxim), 
                 label=r'$\xi_-$ %s' %(cat_dict[cat]['label']),
                 ls='dotted',color=cat_dict[cat]['colour'])
    ax3.axvspan(0,20,color='gray', alpha=0.3)
    # ax3.axvspan(100,200,color='gray', alpha=0.3)
    
ax3.text(0.85, 0.88, '1,1', transform=ax3.transAxes,
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round', pad=0.5))
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel(rf'$\theta$ [{sep_units}]')
ax3.set_xlim([0,200])
ax3.legend(loc='lower left')
_ = ax3.set_ylabel(r'$\xi_-(\theta)$')
# plt.savefig('../plots/xi_minus_%s.pdf' %cat_dict[cat]['label'],bbox_inches='tight')
plt.show()

## Comparison with theory PyCCL

In [ ]:
nz = np.loadtxt('/feynman/work/dap/lcs/lg268561/UNIONS/Catalogues/v1.0/nz/dndz_SP_v1.0_A.txt', usecols=1)
bins = np.loadtxt('/feynman/work/dap/lcs/lg268561/UNIONS/Catalogues/v1.0/nz/dndz_SP_v1.0_A.txt', usecols=0)

def theory_cls(Omega_c, Omega_b, h,n_s,sigma_8):
        #Set cosmology
    cosmo = ccl.Cosmology(Omega_c, Omega_b, h, n_s,sigma_8)

    ell = np.arange(2, 2000)
    theta_deg = np.logspace(np.log10(theta_min/60), np.log10(theta_max/60), num=20) # Theta is in degrees
    #CALCULATION OF THEORY XI_PM
    xi_plus_lf = []
    xi_minus_lf = []

    for i in range(len(nz)):
        bias_ia = 0*np.ones_like(bins[i][:-1])
        lens_ia = ccl.WeakLensingTracer(cosmo, dndz =(np.array(bins[i][:-1]), np.array(nz[i])), ia_bias = (np.array(bins[i][:-1]),bias_ia))
        cl = ccl.angular_cl(cosmo, lens_ia, lens_ia, ell)

        xi_plus_lf.append(list(ccl.correlation(cosmo, ell, cl, theta_deg, type='GG+', method='FFTLog')))
        xi_minus_lf.append(list(ccl.correlation(cosmo, ell, cl, theta_deg, type='GG-', method='FFTLog')))


    style = [':','--','-.']
    plt.errorbar(gg.meanr, gg.xip, yerr=np.sqrt(gg.varxip), ls='', label=r'$\xi_+$ TreeCorr (LF)',capsize=5,marker='o',color='b')
    plt.errorbar(gg.meanr, gg.xim, yerr=np.sqrt(gg.varxim), ls='', label=r'$\xi_-$ TreeCorr (LF)',capsize=5,marker='o',color='g')


    for i in range(len(blinds)):
        plt.plot(theta_deg*60,xi_plus_lf[i],color='b', ls=style[i], label=r'$\xi_+$ PyCCL (LF) blind %s' %blinds[i])
        plt.plot(theta_deg*60,xi_minus_lf[i],color='g', ls=style[i], label=r'$\xi_-$ PyCCL (LF) blind %s' %blinds[i])

    plt.xscale('log')
    # plt.yscale('log')
    plt.legend(fontsize=20)
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.xlim([1,200])
    plt.ylim([0,10e-5])
    plt.ylabel(r'$\xi_\pm(\theta)$')
    plt.xlabel(r'$\theta$ [arcmin]')
    # plt.savefig('plots/pyccl_comparison_lensfit.pdf')



interact(theory_cls,
        Omega_c=widgets.FloatSlider(value=0.26,
                                               min=0.01,
                                               max=0.5,
                                               step=0.01,
                                  description=r'$\Omega_c$'),
        Omega_b=widgets.FloatSlider(value=0.04,
                                               min=0.001,
                                               max=0.07,
                                               step=0.001,
                                  description=r'$\Omega_b$'),
        h=widgets.FloatSlider(value=0.7,
                                               min=0.3,
                                               max=0.9,
                                               step=0.01,
                                  description=r'$h$'),

        n_s=widgets.FloatSlider(value=0.96,
                                               min=0.6,
                                               max=1.1,
                                               step=0.01,
                                  description=r'$n_s$'),
        sigma_8=widgets.FloatSlider(value=0.8,
                                               min=0.3,
                                               max=1.2,
                                               step=0.01,
                                  description=r'$\sigma_8$'))


## Plot varxipm's
Error bars are computed by treecorr, either through the 'shot' or 'jackknife' method.

In [ ]:
for idx,cat in enumerate(cat_options):
    
    blind = 'A'
    label = 'SP_v1.4'
    
    cc = '/n23data1/n06data/lgoh/scratch/CFIS-UNIONS/CFIS-UNIONS_dev/cosmo_inference/data/{}/covs/cov_{}'.format(label+ '_{}'.format(blind),label)
    
    cc_var = np.diag(np.loadtxt(cc +'.txt'))
    cc_varxip = cc_var[:20]
    cc_varxim = cc_var[20:]

    cc_var = np.diag(np.loadtxt(cc+'_g.txt'))
    cc_varxip_g = cc_var[:20]
    cc_varxim_g = cc_var[20:]

    plt.loglog(cat_ggs[idx].meanr,cat_ggs[idx].varxip,'-k', 
               label=r'$\sigma(\xi_+)$ TreeCorr jackknife %s' %cat_dict[cat]['label'])
    plt.loglog(cat_ggs[idx].meanr,cc_varxip, ls='--', c='%s' %cat_dict[cat]['colour'], 
               label=r'$\sigma(\xi_+)$ CosmoCov %s' %cat_dict[cat]['label'])
    plt.loglog(cat_ggs[idx].meanr,cc_varxip_g, ls=':', c='%s' %cat_dict[cat]['colour'], 
               label=r'$\sigma(\xi_+)$ CosmoCov Gaussian %s' %cat_dict[cat]['label'])
    plt.grid()
    plt.xlim([cat_ggs[idx].meanr[0],cat_ggs[idx].meanr[-1]])
    plt.legend(fontsize=15)
    plt.xlabel(rf'$\theta$ [{sep_units}]')
    plt.ylabel(r'$\sigma(\xi_+)$')
    plt.show()
    # plt.savefig()

    plt.loglog(cat_ggs[idx].meanr,cat_ggs[idx].varxim,'-k', 
               label=r'$\sigma(\xi_-)$ TreeCorr jackknife %s' %cat_dict[cat]['label'])
    plt.loglog(cat_ggs[idx].meanr,cc_varxim, ls='--', c='%s' %cat_dict[cat]['colour'], 
               label=r'$\sigma(\xi_-)$ CosmoCov (SP) %s' %cat_dict[cat]['label'])
    plt.loglog(cat_ggs[idx].meanr,cc_varxim_g, ls=':', c='%s' %cat_dict[cat]['colour'], 
               label=r'$\sigma(\xi_-)$ CosmoCov (SP) Gaussian %s' %cat_dict[cat]['label'])
    plt.grid()
    plt.xlim([cat_ggs[idx].meanr[0],cat_ggs[idx].meanr[-1]])
    plt.legend(fontsize=15)
    plt.xlabel(rf'$\theta$ [{sep_units}]')
    plt.ylabel(r'$\sigma(\xi_-)$')
    plt.show()
    # plt.savefig()



## Run systematic tests

### C_sys

In [ ]:
#CALCULATE XI_SYS FOR SHAPEPIPE

sep_units = 'arcmin'
theta_min = 1
theta_max = 200

TreeCorrConfig = {
        'ra_units': 'degrees',
        'dec_units': 'degrees',
        'max_sep': str(theta_max),
        'min_sep': str(theta_min),
        'sep_units': sep_units,
        'nbins': 20,
        'var_method':'jackknife',
    }

with fits.open(cat_dict[11]['dir']) as data:
    df_psf = pd.DataFrame(data[1].data)
            
cat_psf = treecorr.Catalog(
    ra=df_psf['RA'],
    dec=df_psf['DEC'],
    g1=df_psf['E1_PSF_HSM'],
    g2=df_psf['E2_PSF_HSM'],
    ra_units='degrees',
    dec_units='degrees',
    npatch=50
)

gg_psf = treecorr.GGCorrelation(TreeCorrConfig)
gg_psf.process(cat_psf)

ggs_psf_star = []
for idx,cat in enumerate(cat_options):
    cat_gal = treecorr.Catalog(
        ra=dfs[idx]['ra'],
        dec=dfs[idx]['dec'],
        g1=dfs[idx]['e1']-cat_dict[cat]['e1_bias'],
        g2=dfs[idx]['e2']-cat_dict[cat]['e2_bias'],
        w=dfs[idx]['w'],
        ra_units='degrees',
        dec_units='degrees',
        npatch=50
    )
    gg_psf_star = treecorr.GGCorrelation(TreeCorrConfig)
    gg_psf_star.process(cat_gal,cat_psf)
    ggs_psf_star.append(gg_psf_star)    

    print("done for cat %s" %cat)

In [ ]:
for idx,cat in enumerate(cat_options):
    
    C_sys_xip = gg_psf.xip
    C_sys_xim = gg_psf.xim

    # delta_C_sys_xip = C_sys_xip*np.sqrt((2*np.sqrt(ggs_psf_star[idx].varxip)/ggs_psf_star[idx].xip)**2+(np.sqrt(gg_psf.varxip)/gg_psf.xip)**2)
    # delta_C_sys_xim = C_sys_xim*np.sqrt((2*np.sqrt(ggs_psf_star[idx].varxim)/ggs_psf_star[idx].xim)**2+(np.sqrt(gg_psf.varxim)/gg_psf.xim)**2)
    
    plt.errorbar(ggs_psf_star[idx].meanr, C_sys_xip, yerr=0, 
                 label=r'$(\xi^{sys}_+)$ Catalogue %s'%cat_dict[cat]['label'],
                 ls=cat_dict[cat]['ls'],color=cat_dict[cat]['colour'])
    plt.legend()
    plt.xlabel(r'$\theta[arcmin]$')
    plt.ylabel(r'$\xi^{sys}_\pm$')
    # plt.ylim([-2e-8,2e-8])
    plt.xscale('log')
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.grid(True)
    
    # plt.errorbar(ggs_psf_star[idx].meanr, C_sys_xim, yerr=delta_C_sys_xim, label=r'$(\xi^{sys}_-)$ Catalogue %s'%cat_dict[cat]['label'],color='g')
    # plt.legend()
    # plt.xlabel(r'$\theta[arcmin]$')
    # plt.ylabel(r'$\xi^{sys}_\pm$')
    # plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    # # plt.ylim([-2e-8,2e-8])
    # plt.xscale('log')
    # plt.grid(True)

### M_ap

In [ ]:
for idx,cat in enumerate(cat_options):
    R = cat_ggs[idx].rnom

    (Map_lf,mapsq_im_lf,Mx_lf,mxsq_im_lf,varMapsq_lf) = cat_ggs[idx].calculateMapSq(R=R,m2_uform='Schneider')
    (Map_sp,mapsq_im_sp,Mx_sp,mxsq_im_sp,varMapsq_sp) = cat_ggs[idx].calculateMapSq(R=R,m2_uform='Schneider')
    
    plt.errorbar(R, Map_lf, yerr=np.sqrt(varMapsq_lf), label=r'$<M^2_{ap}>$ {}'.format(cat_dict[cat]['label']),ls=':',color='b')
    plt.errorbar(R, Mx_lf, yerr=np.sqrt(varMapsq_lf), label=r'$<M^2_{x}>$ {}'.format(cat_dict[cat]['label']),ls=':',color='r')
    plt.axhline(y=0,xmin=0,xmax=200,color='k')
    plt.xlabel(r'$\theta[arcmin]$')
    plt.ylabel(r'$<M^2>$')
    plt.xscale('log')
    plt.ylim([-2e-5,1e-5])
    # plt.xlim([1,200])
    plt.grid(True)
    plt.legend()
    
    plt.errorbar(R, Map_sp, yerr=np.sqrt(varMapsq_sp), label=r'$<M^2_{ap}>$ {}'.format(cat_dict[cat]['label']),ls=':',color='b')
    plt.errorbar(R, Mx_sp, yerr=np.sqrt(varMapsq_sp), label=r'$<M^2_{x}>$ {}'.format(cat_dict[cat]['label']),ls=':',color='r')
    plt.axhline(y=0,xmin=0,xmax=200,color='k')
    plt.xscale('log')
    plt.xlabel(r'$\theta[arcmin]$')
    plt.ylabel(r'$<M^2>$')
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.ylim([-2e-5,1e-5])
    plt.grid(True)
    plt.legend()

## Plot Covariance Matrix

In [ ]:
from numpy import linalg as LA

%matplotlib inline

def get_cov(filename):

	data = np.loadtxt(filename)
	ndata = int(np.max(data[:,0]))+1

	print("Dimension of cov: %dx%d"%(ndata,ndata))

	# ndata_min = int(np.min(data[:,0]))
	cov_g = np.zeros((ndata,ndata))
	cov_ng = np.zeros((ndata,ndata))
	for i in range(0,data.shape[0]):
		cov_g[int(data[i,0]),int(data[i,1])] =data[i,8]
		cov_g[int(data[i,1]),int(data[i,0])] =data[i,8]
		cov_ng[int(data[i,0]),int(data[i,1])] =data[i,9]
		cov_ng[int(data[i,1]),int(data[i,0])] =data[i,9]

	return cov_g, cov_ng, ndata


covfile = '/feynman/work/dap/lcs/lg268561/UNIONS/CFIS-UNIONS/CFIS-UNIONS_dev/cosmo_inference/data/SP_cut_Fabian/covs/out_cov_ssss_+-_cov_Ntheta20_Ntomo1_3'

c_g, c_ng, ndata = get_cov(covfile)	
cov = c_ng+c_g
cov_g = c_g

b = np.sort(LA.eigvals(cov))
print("min+max eigenvalues cov: %e, %e"%(np.min(b), np.max(b)))
if(np.min(b)<=0.):
    print("non-positive eigenvalue encountered! Covariance Invalid!")
    exit()

print("Covariance is postive definite!")

pp_var = []
for i in range(ndata):

    pp_var.append(cov[i][i])
    
    
cmap = 'seismic'

pp_norm = np.zeros((ndata,ndata))
for i in range(ndata):
    for j in range(ndata):
        pp_norm[i][j] = cov[i][j]/ np.sqrt(cov[i][i]*cov[j][j])

print("Plotting correlation matrix ...")

# plot_path = covfile+'_plot.pdf'

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.xaxis.tick_top()
ax.xaxis.set_ticks(np.arange(0, 41, 1))
ax.yaxis.set_ticks(np.arange(0, 41, 1))


plt.axvline(x=19.5,color='black',linewidth=1.5)
plt.axhline(y=19.5,color='black',linewidth=1.5)


im3 = ax.imshow(pp_norm, cmap=cmap, vmin=-1, vmax=1)
ax.get_xaxis().set_ticklabels([])
ax.get_yaxis().set_ticklabels([])
cbar = fig.colorbar(im3, orientation='vertical',shrink=0.6,ticks=[-1, 0, 1])
cbar.ax.tick_params(labelsize=15)
cbar.ax.set_yticklabels([r'$-1$', r'$0$', r'$1$']) 

ax.text(8, -2, r'$\xi_+^{ij}(\theta)$', fontsize=22)
ax.text(30, -2, r'$\xi_-^{ij}(\theta)$', fontsize=22)
ax.text(-6, 10, r'$\xi_+^{ij}(\theta)$', fontsize=22)
ax.text(-6, 30, r'$\xi_-^{ij}(\theta)$', fontsize=22)
# ax.set_title('Blind A',fontsize=15)

plt.savefig('../plots/unions_covmat.pdf', bbox_inches='tight')



plt.show()
# print("Plot saved as %s"%(plot_path))
